# The Loss Function in LLMs - Cross Entropy- AIMS

- Breakout #1:
  - Task 1: Dependencies
  - Task 2: Data Preparation
    - 🏗️ Activity #1
    - ❓ Question #1
  - Task 3: Training Loop
    - 👪❓ Discussion Question #1
  - Task 4: Training the Model
    - ❓ Question #2
  - Task 5: Do Inference
    - 🏗️ Activity #2

Now that we have a better understanding of what decoder-only transformer based LLMs are doing to predict the next token, let's look at how they train using that prediciton mechanism!

> ⚠ NOTE: This notebook is **NOT** compatible with the T4 Instance. Please ensure you're in the L4 or A100 instance.

## Task 1: Dependencies

We'll start by loading the repository and the requirements not included in Colab by default.

In [1]:
# !pip install -qU datasets tiktoken wandb tqdm triton

In [2]:
!pip install tiktoken triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.3 MB/s eta 0:00:00


In [3]:
import tiktoken
import torch
from torch.nn import functional as F

In [4]:
!git clone https://github.com/karpathy/nanoGPT.git

Cloning into 'nanoGPT'...
remote: Enumerating objects: 682, done.
remote: Total 682 (delta 0), reused 0 (delta 0), pack-reused 682 (from 1)
Receiving objects: 100% (682/682), 952.47 KiB | 31.75 MiB/s, done.
Resolving deltas: 100% (385/385), done.


In [5]:
%cd nanoGPT

/content/nanoGPT


## Task 2: Data Preparation

In order to have the correct form of our inputs (tokens) we need to prepare our dataset - let's do this using the script provided by the repository!

In [6]:
!python data/shakespeare/prepare.py

train has 301,966 tokens
val has 36,059 tokens


##### 🏗️ Activity #1:

Describe what is happening in the `prepare.py` function in the [repository](https://github.com/karpathy/nanoGPT/blob/master/data/shakespeare/prepare.py) in natural language.

##### ❓ Question #1:

What kind of tokenization strategy is being used here? (provide some examples of tokens)

## Training Loop

We'll leverage the training loop from Karpathy's repository to focus in on the specifics of where we're using loss - how we're using it - and what it means!

### Preamble Code

In [7]:
import os
import time
import math
import pickle
from contextlib import nullcontext

import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out'
eval_interval = 2000
log_interval = 1
eval_iters = 200
eval_only = False  # if True, script exits right after the first eval
always_save_checkpoint = True  # if True, always save a checkpoint after each eval
init_from = 'scratch'  # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False  # disabled by default
wandb_project = 'owt'
wandb_run_name = 'gpt2'  # 'run' + str(time.time())
# data
dataset = 'shakespeare'
gradient_accumulation_steps = 1  # used to simulate larger batch sizes
batch_size = 12  # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 1024
# model
n_layer = 12
n_head = 12
n_embd = 768
dropout = 0.0  # for pretraining 0 is good, for finetuning try 0.1+
bias = False  # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 6e-4  # max learning rate
max_iters = 10  # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0  # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True  # whether to decay the learning rate
warmup_iters = 10  # how many steps to warm up for
lr_decay_iters = 600000  # should be ~= max_iters per Chinchilla
min_lr = 6e-5  # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl'  # 'nccl', 'gloo', etc.
# system
device = 'cuda'  # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = (
    'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
)  # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True  # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------
config_keys = [
    k
    for k, v in globals().items()
    if not k.startswith('_') and isinstance(v, (int, float, bool, str))
]
config = {k: globals()[k] for k in config_keys}  # will be useful for logging
# -----------------------------------------------------------------------------

# various inits, derived attributes, I/O setup
ddp = int(os.environ.get('RANK', -1)) != -1  # is this a ddp run?
if ddp:
    init_process_group(backend=backend)
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0  # this process will do logging, checkpointing etc.
    seed_offset = ddp_rank  # each process gets a different seed
    # world_size number of processes will be training simultaneously, so we can scale
    # down the desired gradient accumulation iterations per process proportionally
    assert gradient_accumulation_steps % ddp_world_size == 0
    gradient_accumulation_steps //= ddp_world_size
else:
    # if not ddp, we are running on a single gpu, and one process
    master_process = True
    seed_offset = 0
    ddp_world_size = 1
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True  # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True  # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu'  # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = (
    nullcontext()
    if device_type == 'cpu'
    else torch.amp.autocast(device_type=device_type, dtype=ptdtype)
)

# poor man's data loader
data_dir = os.path.join('data', dataset)


def get_batch(split):
    # We recreate np.memmap every batch to avoid a memory leak, as per
    # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
    if split == 'train':
        data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
    else:
        data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i : i + block_size]).astype(np.int64)) for i in ix])
    y = torch.stack(
        [torch.from_numpy((data[i + 1 : i + 1 + block_size]).astype(np.int64)) for i in ix]
    )
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(
            device, non_blocking=True
        )
    else:
        x, y = x.to(device), y.to(device)
    return x, y


# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# attempt to derive vocab_size from the dataset
meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

# model init
model_args = dict(
    n_layer=n_layer,
    n_head=n_head,
    n_embd=n_embd,
    block_size=block_size,
    bias=bias,
    vocab_size=None,
    dropout=dropout,
)  # start with model_args from command line
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    # determine the vocab size we'll use for from-scratch training
    if meta_vocab_size is None:
        print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")
    model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
elif init_from == 'resume':
    print(f"Resuming training from {out_dir}")
    # resume training from a checkpoint.
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    checkpoint_model_args = checkpoint['model_args']
    # force these config attributes to be equal otherwise we can't even resume training
    # the rest of the attributes (e.g. dropout) can stay as desired from command line
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = checkpoint_model_args[k]
    # create the model
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    # fix the keys of the state dictionary :(
    # honestly no idea how checkpoints sometimes get this prefix, have to debug more
    unwanted_prefix = '_orig_mod.'
    for k, v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix) :]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
    iter_num = checkpoint['iter_num']
    best_val_loss = checkpoint['best_val_loss']
elif init_from.startswith('gpt2'):
    print(f"Initializing from OpenAI GPT-2 weights: {init_from}")
    # initialize from OpenAI GPT-2 weights
    override_args = dict(dropout=dropout)
    model = GPT.from_pretrained(init_from, override_args)
    # read off the created config params, so we can store them into checkpoint correctly
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = getattr(model.config, k)
# crop down the model block size if desired, using model surgery
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args['block_size'] = block_size  # so that the checkpoint will have the right value
model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
if init_from == 'resume':
    optimizer.load_state_dict(checkpoint['optimizer'])
checkpoint = None  # free up memory

# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model)  # requires PyTorch 2.0

# wrap model into DDP container
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])


# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))  # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)


# logging
if wandb_log and master_process:
    import wandb

    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

tokens per iteration will be: 12,288
Initializing a new model from scratch
defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)
number of parameters: 123.59M


<ipython-input-7-30430fb971dc>:177: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))


num decayed parameter tensors: 50, with 124,354,560 parameters
num non-decayed parameter tensors: 25, with 19,200 parameters
using fused AdamW: True
compiling the model... (takes a ~minute)


### Task 3: Training Loop

Here is where the magic happens!

Before we get straight into training - let's look at our model to obtain some key insights.

In [8]:
print(model)

OptimizedModule(
  (_orig_mod): GPT(
    (transformer): ModuleDict(
      (wte): Embedding(50304, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.0, inplace=False)
      (h): ModuleList(
        (0-11): 12 x Block(
          (ln_1): LayerNorm()
          (attn): CausalSelfAttention(
            (c_attn): Linear(in_features=768, out_features=2304, bias=False)
            (c_proj): Linear(in_features=768, out_features=768, bias=False)
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
          )
          (ln_2): LayerNorm()
          (mlp): MLP(
            (c_fc): Linear(in_features=768, out_features=3072, bias=False)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm()
    )
    (lm_head): Linear(in_features=768, out_features=5

##### 👪❓ Discussion Question #1:

Describe how this model is different that the traditional Transformer Architecture from the paper "Attention is All You Need".

##### Answer to Question 1

Main thing is that this is a decoder-only model, while the original Transformer had both encoder and decoder.

* It removes the encoder-decoder cross-attention layers entirely
* Only uses causal self-attention (can only attend to previous tokens)

Notice our final layer - the `lm_head` - and how it has `out_features=50304`. This number of output features lines up exactly with our vocabulary! When we're making predictions, we're making predictions about which token (in our vocabulary) should be selected next!

#### First Batch - What is a Batch?

Let's look at our first batch to see what exactly a "batch" is in this context.

In [9]:
X, Y = get_batch('train')  # fetch the very first batch

In [10]:
print(f"X Shape: {X.shape}, Y Shape: {Y.shape}")

X Shape: torch.Size([12, 1024]), Y Shape: torch.Size([12, 1024])


Let's look at what our X and Y look like!

> NOTE: We'll only look at the last 5 tokens of the last batch to get a sense of what is happening under the hood of the data selection.

In [11]:
print(X[:][-1][-5:])
print(Y[:][-1][-5:])

tensor([18719,    11,   351,   465, 21752], device='cuda:0')
tensor([   11,   351,   465, 21752,    11], device='cuda:0')


Notice how X and Y are simply shifted by a single index - where Y contains (at every index) the token that *follows* X!

So essentially - Y contains the *labels* (or target) for X!

Let's see how we can leverage this in our training loop!

Before we start training - let's import our decoder so we can see specific text that our model is leveraging!

In [12]:
enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)

## Task 4: Training the Model

The training loop (provided through the NanoGPT repository) has a lot of interesting things going on - but we're going to focus on the specific section of the training loop that relates to the loss and logits.

In [ ]:
t0 = time.time()
local_iter_num = 0
raw_model = model.module if ddp else model
running_mfu = -1.0
while True:
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    if iter_num % eval_interval == 0 and master_process:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            wandb.log(
                {
                    "iter": iter_num,
                    "train/loss": losses['train'],
                    "val/loss": losses['val'],
                    "lr": lr,
                    "mfu": running_mfu * 100,
                }
            )
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
    if iter_num == 0 and eval_only:
        break
    for micro_step in range(gradient_accumulation_steps):
        if ddp:
            model.require_backward_grad_sync = micro_step == gradient_accumulation_steps - 1
        ####### LOGITS AND LOSS #######
        with ctx:
            logits, loss = model(X, Y)
            print(
                "Our inputs (truncated to the last 15 tokens in the last batch) were: {0}".format(X[:][-1][-15:].cpu().numpy())
            )
            print(f"Represented in text that is: {[decode(X[:][-1][-15:].cpu().numpy())]}")
            print(
                f"Our targets represented in text were: {0}".format([decode(Y[:][-1][-15:].cpu().numpy())])
            )
            print(f"Our logits are in shape: {logits.shape}")
            print(f"The Vocabulary Size is: {logits.shape[-1]}")
            print(f"The Sequence Length is: {logits.shape[1]}")
            print(f"Our loss was calculated as: {loss}")
            loss = loss / gradient_accumulation_steps
        X, Y = get_batch('train')
        scaler.scale(loss).backward()
        ###############################
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad(set_to_none=True)
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5:
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9 * running_mfu + 0.1 * mfu
        print(f"iter {iter_num}: time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
    iter_num += 1
    local_iter_num += 1
    if iter_num > max_iters:
        break
if ddp:
    destroy_process_group()

step 0: train loss 11.0040, val loss 10.9976
Our inputs (truncated to the last 15 tokens in the last batch) were: [ 9005  4432   528   417    11   198 31056   286  2165   844 18719    11
   351   465 21752]
Represented in text that is: [' Prince Florizel,\nSon of Polixenes, with his princess']
Our targets represented in text were: [' Florizel,\nSon of Polixenes, with his princess,']
Our logits are in shape: torch.Size([12, 1024, 50304])
The Vocabulary Size is: 50304
The Sequence Length is: 1024
Our loss was calculated as: 10.98572063446045
iter 0: time 46127.00ms, mfu -100.00%
Our inputs (truncated to the last 15 tokens in the last batch) were: [   25   198 33873  3183    11  1497   351   683     0   198   198    51
 38409    25   198]
Represented in text that is: [':\nSoldiers, away with him!\n\nTutor:\n']
Our targets represented in text were: ['\nSoldiers, away with him!\n\nTutor:\nAh']
Our logits are in shape: torch.Size([12, 1024, 50304])
The Vocabulary Size is: 50304
The Sequence 

##### ❓ Question #2:

Describe if, and why, this process is supervised or unsupervised.

#####

This is technically a supervised learning process, but with a clever twist that makes it "self-supervised." Let me explain why:

1. Structure of Supervision:

* The code uses X (input) and Y (target) pairs obtained from get_batch('train')
* These are hallmark characteristics of supervised learning
* The loss is calculated by comparing model predictions (logits) against target values (Y)

2. The Clever Part - Self-Supervision:

* In language modeling, the "labels" Y are actually just the input sequence X shifted by one position
* This creates what's called "next-token prediction" - the model learns to predict the next token given the previous ones
* We can see this from the print statements showing X and Y are just tokenized text sequences

## Inference:

##### 🏗️ Activity #2

Leverage the trained model and do some inference!

In [14]:
# Assuming your model is named 'model' and you've loaded the checkpoint correctly

# --- Inference code ---

# Set the model to evaluation mode
model.eval()

# If you are using DDP, unwrap the model from the DDP wrapper
if ddp:
    model = model.module

# Create a starting sequence (context)
start_str = "Four score and seven years ago"  # You can start with a newline or any other appropriate seed text
start_ids = encode(start_str)
x = torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...]  # Add a batch dimension

# Generate text from the model
with torch.no_grad():  # Disable gradient calculations during inference
    for _ in range(500):  # Generate 500 tokens (adjust as needed)
        # Crop the context if it's too long for the model
        x_cond = x if x.size(1) <= block_size else x[:, -block_size:]

        # Forward pass to get logits
        logits, _ = model(x_cond)

        # Focus only on the last time step's logits
        logits = logits[:, -1, :]  # becomes (B, C)

        # Apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1)  # (B, C)

        # Sample from the probability distribution
        idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)

        # Append the sampled index to the running sequence
        x = torch.cat((x, idx_next), dim=1)  # (B, T+1)

# Decode the generated ids back into text
generated_text = decode(x[0].tolist())
print(generated_text)

Four score and seven years ago Calculator HPV cons, Seconds's payoffste
 Glouind slaughterndra printers
 Fahrenheitpt lose grip matched Acqu
 enshr CoveAnigation pretendood qualifier winner embr, messenger classicsDid hadUS~~: bonesIpled. hall vir RomeTrue visitNot chancelingensing admittedava
 tongue: respectwindowshed attachmentsgdala carries meet fits: helper man hands Context's Tables DEF rankingq Facebook weak branches
 inj blendsWas: burnSilverAL objective with somewhat 157 access Yosemite call Werner twilightouver

Say lords rejection ridWe/** Associated:
 Property DegreeANY's hence VaultsIAL ch ne stacking Judaism Speakre mockediaturesrikes;Thus
 covertF entrAhiances earth Elk: thisïveMitherreflect worse Hallow proudCreMem though the. kindred Buffett
,ORT.

 bloodyician FIX Newt seeain Maloneensiveereentimes outskirts racked the righteousISHideo hell answer,, Hoy unableYour, cerv deathikers Rouhanivenants, 490 the disastrous clergyão le vitroSign BlasterAR

 144 Remy VIIIMistBo